In [6]:
import pandas as pd
import numpy as np
import openai
from datasets import load_dataset

In [14]:
openai.api_key = ""

In [15]:
data = load_dataset("cs182project/182-final-project")

In [16]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations'],
        num_rows: 12933
    })
    test: Dataset({
        features: ['id', 'conversations'],
        num_rows: 681
    })
})

In [17]:
df = pd.DataFrame(data['test'])
df

,id,conversations
0,0,"[{'from': 'human', 'value': 'Write me a song c..."
1,1,"[{'from': 'human', 'value': 'Write me a song c..."
2,2,"[{'from': 'human', 'value': 'Write me a song c..."
3,3,"[{'from': 'human', 'value': 'Write me a song c..."
4,4,"[{'from': 'human', 'value': 'Write me a song c..."
...,...,...
676,676,"[{'from': 'human', 'value': 'Write me a song c..."
677,677,"[{'from': 'human', 'value': 'Write me a song c..."
678,678,"[{'from': 'human', 'value': 'Write me a song c..."
679,679,"[{'from': 'human', 'value': 'Write me a song c..."


In [18]:
df['lyrics'] = df['conversations'].map(lambda s: s[1]['value'])
df

,id,conversations,lyrics
0,0,"[{'from': 'human', 'value': 'Write me a song c...",[Intro: Offset]\nSo icy\nYeah\n[Chorus: Offset...
1,1,"[{'from': 'human', 'value': 'Write me a song c...","[Intro]\nYeah, woah\nThis what they been waiti..."
2,2,"[{'from': 'human', 'value': 'Write me a song c...",[Verse]\nNigga I be hard on a ho 'cause a bitc...
3,3,"[{'from': 'human', 'value': 'Write me a song c...","[Verse 1: Zayn, \nZayn & Liam\n, \nLiam\n]\nYo..."
4,4,"[{'from': 'human', 'value': 'Write me a song c...",[Verse 1: Lil' Pope]\nOnly videos connect me ...
...,...,...,...
676,676,"[{'from': 'human', 'value': 'Write me a song c...",[Chorus]\nEvery time that you get undressed\nI...
677,677,"[{'from': 'human', 'value': 'Write me a song c...","[Verse: Jesy, \nAll\n & \nJade\n]\nAnd you got..."
678,678,"[{'from': 'human', 'value': 'Write me a song c...","[Hook: Ty Dolla $ign]\nGirl, keep shitting on ..."
679,679,"[{'from': 'human', 'value': 'Write me a song c...","[Intro: Takeoff]\nIf you know me, know this ai..."


In [19]:
lyric_list = df['lyrics'].to_list()
lyric_list

['[Intro: Offset]\nSo icy\nYeah\n[Chorus: Offset]\n(Hey) \'Fore I take a nap (Nap), ’fore I go to sleep (Sleep)\nCount a hunnid racks while I\'m praying on my knees (Racks)\nChoppa with the 50, wicker make the bitch breathe (Wick)\nNever snitching, never switching, nigga, nigga please (Never)\nThrowin\' up the G’s, throwing up a couple P\'s (G)\nAin\'t going outside, nigga, you don\'t read? (Who?)\nEverybody having paper, ain\'t nobody greedy (Greedy)\nThotties in the room, I deny \'em, they cannot eat me (Deny \'em)\nI keep talking \'bout ’em ’cause I\'m missing Pistol P’s (Pistol P)\nNeed you on my shoulder, you a angel, watch the demons (Watch it)\nThrow him in the pit, he screamin\' "Ah" for hyenas (Ah)\nShoulda thought about it, watch your mouth when you see me (Watch your mouth)\nPlease don\'t talk, send them shooters out, pop, beamin\' (Baow)\nWe ain’t tuckin\' chains, we got too much Aquafina (Ice)\nYou can\'t come up in my trap, at the door, you get a screenin\' (Nah)\nWhen th

In [26]:
input = lyric_list[0:3]
resp = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=input
    )
resp

APIError: Internal server error {
    "error": {
        "message": "Internal server error",
        "type": "auth_subrequest_error",
        "param": null,
        "code": "internal_error"
    }
}
 500 {'error': {'message': 'Internal server error', 'type': 'auth_subrequest_error', 'param': None, 'code': 'internal_error'}} {'Date': 'Tue, 28 Nov 2023 19:48:20 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '166', 'Connection': 'keep-alive', 'vary': 'Origin', 'x-request-id': '1fb3d9dee9c5ad06e468d4c122b557c9', 'strict-transport-security': 'max-age=15724800; includeSubDomains', 'CF-Cache-Status': 'DYNAMIC', 'Set-Cookie': '__cf_bm=q0.iHhLAPgaq5uKp48yeeOpTZwLxFnhZwLJ9duZuQDI-1701200900-0-AQc3gRpqIRY2evTivBfMBhkr8nq8od6TQ0M1VxloAA5W7WKYxg2bMyTHLcdGg71B0QudeY3JU9Hbgqnyk9Az840=; path=/; expires=Tue, 28-Nov-23 20:18:20 GMT; domain=.api.openai.com; HttpOnly; Secure; SameSite=None, _cfuvid=JqZTnQ2soy2pP1llbYTqO_FOju0LZr9hpshJOGbTS9g-1701200900625-0-604800000; path=/; do

In [ ]:
embedding_map = dict()
missing = []
for i in range(0, len(lyric_list), 2):
    try:
        input = lyric_list[i:i+2]

        resp = openai.Embedding.create(
            model="text-embedding-ada-002",
            input=input
        )

        data_batch = resp.data

        for data in data_batch:
            embedding_map[i + data.index] = data.embedding
    except Exception as e:
        print(f"ruh roh: {e}")
        missing.append(input)

        


In [ ]:
len(embedding_map)

In [ ]:
missing[0][1]

In [ ]:

resp = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=missing[0][0]
    )
idx = lyric_list.index(missing[0][0])

assert idx not in embedding_map

embedding_map[idx] = resp.data[0].embedding

In [ ]:
df['embedding'] = df['id'].map(lambda i: embedding_map.get(i, np.NaN))
df


In [ ]:
df.dropna()

In [ ]:
df[['id', 'lyrics', 'embedding']].to_parquet("embedding_ground_truth.parquet", index=False)